<a href="https://colab.research.google.com/github/simulate111/machine-learning-aalto/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle
!pip install xgboost
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d lainguyn123/student-performance-factors
!unzip student-performance-factors.zip

Dataset URL: https://www.kaggle.com/datasets/lainguyn123/student-performance-factors
License(s): CC0-1.0
student-performance-factors.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  student-performance-factors.zip
replace StudentPerformanceFactors.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd

df = pd.read_csv('StudentPerformanceFactors.csv')  # Replace with the actual filename
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
# input and output
features = df[['Hours_Studied', 'Sleep_Hours', 'Attendance', 'Teacher_Quality']]
target = df['Exam_Score']

In [ ]:
# Ecategorical One-hot encoding
features = pd.get_dummies(features, drop_first=True)
#Data splitting
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

#train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

#Generalization
y_pred = rf_model.predict(X_test)

# Model evaluation
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

In [ ]:
#R-squared and handle the case with fewer than two samples


# Step 8: Define the Huber Loss function
def huber_loss(y_true, y_pred, delta=1.0):
    error = y_true - y_pred
    is_small_error = abs(error) <= delta
    squared_loss = 0.5 * error**2
    linear_loss = delta * (abs(error) - 0.5 * delta)
    return np.where(is_small_error, squared_loss, linear_loss).mean()

#Huber Loss
huber = huber_loss(y_test, y_pred, delta=1.0)

print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Huber Loss: {huber:.2f}")
print(f"R-squared: {r2:.2f}")

In [ ]:
#Data Visualize
plt.figure(figsize=(14, 10))
plt.subplot(2, 2, 1)
sns.histplot(df['Hours_Studied'], bins=10, color='blue')
plt.title('Distribution of Hours Studied')

plt.subplot(2, 2, 2)
sns.histplot(df['Sleep_Hours'], bins=10, color='green')
plt.title('Distribution of Sleep Hours')

plt.subplot(2, 2, 3)
sns.histplot(df['Attendance'], bins=10, color='purple')
plt.title('Distribution of Attendance')

plt.subplot(2, 2, 4)
sns.countplot(x='Teacher_Quality', data=df, palette='Set2')
plt.title('Count of Teacher Quality')

plt.tight_layout()
plt.show()

In [ ]:
#Categorical  to Numeric
df_encoded = pd.get_dummies(df, drop_first=True)

# Correlation Matrix
plt.figure(figsize=(16, 12))
sns.heatmap(df_encoded.corr(), annot=True, cmap='coolwarm', fmt='.2f', annot_kws={"size": 10})
plt.title('Correlation Matrix', fontsize=18)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.yticks(rotation=0, fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Predicted vs. Actual Values
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, color='blue', edgecolor='k')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.xlabel('Actual Exam Scores')
plt.ylabel('Predicted Exam Scores')
plt.title('Actual vs Predicted Exam Scores')
plt.show()

In [ ]:
#Feature Importance
importances = rf_model.feature_importances_
feature_importances = pd.DataFrame({'Feature': features.columns, 'Importance': importances})

plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importances.sort_values(by='Importance', ascending=False), palette='viridis')
plt.title('Feature Importances from Random Forest Model')
plt.show()